In [1]:
##Instalação necessaria

In [ ]:
!pip -q install langchain huggingface_hub tiktoken
!pip -q install chromadb
!pip -q install PyPDF2 pypdf sentence_transformers
!pip -q install -U FlagEmbedding

In [3]:
#Configuração

In [4]:
import os
import torch 

In [5]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [7]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)

cpu


In [8]:
#Importações 

In [9]:
import gc
import textwrap
from peft import LoraConfig, get_peft_model
from langchain.vectorstores import Chroma
from langchain.schema import prompt
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from typing import Any, List, Mapping, Optional

/home/maria.carreiro/.conda/envs/eduarda2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
model_name = "/scratch/Llama-2-13b-chat-hf"
#Caminho do modelo que voce pretende utilizar 

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto"
)
model.config.use_cache = False

Loading checkpoint shards: 100%|██████████| 3/3 [04:00<00:00, 80.02s/it] 


In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, is_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
lora_config = LoraConfig.from_pretrained('/scratch/......')
model = get_peft_model(model, lora_config)

/home/maria.carreiro/.conda/envs/eduarda2/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/maria.carreiro/.conda/envs/eduarda2/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [14]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [15]:
class LlamaLLM(LLM):
    model = model

    temperature: float = 0.1

    max_new_tokens: int = 350

    @property
    def _llm_type(self) -> str:
        return "llama"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        with torch.no_grad():
#             inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
            inputs = tokenizer(prompt, return_tensors="pt")

            generation_output = model.generate(

                    input_ids=inputs.input_ids,
                    return_dict_in_generate=True,
                    output_scores=True,
                    max_new_tokens=self.max_new_tokens,
                    temperature=self.temperature,
                )
            generated_text = tokenizer.decode(generation_output.sequences[0], skip_special_tokens=True)
            generated_text = generated_text.split("[/INST]")[1]
        return generated_text

In [16]:
#Gerador de Embeddings

In [ ]:
#gera embeddings
model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True} #Definir como 'True' para computar similaridade de cosseno

with torch.no_grad():
    model_norm = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs={'device': 'auto'},
        encode_kwargs=encode_kwargs
    )

embedding_function = model_norm

In [ ]:
#Carregamento de vector database

In [ ]:
#chama o banco criado
vet_db = Chroma(persist_directory="/scratch/caminho do seu banco de dados", embedding_function=embedding_function)

In [ ]:
## Retrieval-Augmented Generation (RAG), criação dos prompts e geração dos textos

In [ ]:
#Cria um Retriever
retriever = vet_db.as_retriever(search_kwargs={"k": 1})

In [ ]:
#cria o prompt de default
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
Você é um atendende de vendas. Suas respostas não devem incluir conteúdo prejudicial, antiético, racista, sexista, tóxico, perigoso ou ilegal.
Suas respostas devem responder à pergunta apenas uma vez e não conter nenhum texto após a resposta ser feita.
Se uma pergunta não fizer sentido ou não for  coerente, explique o porquê, em vez de responder algo incorreto. 
Se você não sabe a resposta a uma pergunta, não compartilhe informações falsas."""
#Coloque o prompt ideal de acordo com sua demanda 

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

In [ ]:
del _

In [ ]:
gc.collect()

In [ ]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
llm = LlamaLLM(
    model= model,
    temperature = 0.1,
    max_new_tokens = 350,
    callback_manager=callback_manager,
    verbose=True,
)

In [ ]:
prompt_template = get_prompt(instruction, sys_prompt)

llama_prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
chain_type_kwargs = {"prompt": llama_prompt}

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="stuff",
                                       retriever=retriever,
                                       chain_type_kwargs=chain_type_kwargs,
                                       return_source_documents=True
                                      )

In [ ]:
## Cita as fontes da resposta

def wrap_text_preserve_newlines(text, width=110):
    lines = text.split('\n')

    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nFonte:')
    for source in llm_response["source_documents"]:
        print("IdDocumentoField: ", source.metadata['source'])

In [ ]:
torch.cuda.empty_cache()

In [ ]:
query = "Prompt Adequado"
process_llm_response(llm_response)

In [ ]:
query = "Prompt Adequado"
process_llm_response(llm_response)

In [ ]:
query = "Prompt Adequado"
process_llm_response(llm_response)